In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import re
import os

In [ ]:
SITENAME = "rottentomatoes"
SITEMAP_SEED = 'https://www.rottentomatoes.com/sitemap_64.xml'
PAGES_SAVE_DIR = "pages-rotten"
TABLES_SAVE_DIR = "tables-rotten"
TEMPLATE_DEEP = 3

In [ ]:
r = requests.get(SITEMAP_SEED) #URL della sitemap del sito target
xml = r.text
soup = BeautifulSoup(xml)
sitemapTags = soup.find_all("loc")
pages = np.array([])

for page in sitemapTags: 
    pages  = np.append(pages, page.string)

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 100


df = pd.DataFrame(data=pages)
df = df.loc[df[0].str.contains("https://www\..+\.\w+(/[^/]+){"+str((TEMPLATE_DEEP+1))+"}$" , regex = True)]

df.shape

In [ ]:
page_dir = ".\\"+PAGES_SAVE_DIR+"\\"
if not os.path.exists(page_dir):
    os.makedirs(page_dir)
    
t_page_dir = ".\\"+TABLES_SAVE_DIR+"\\"
if not os.path.exists(t_page_dir):
    os.makedirs(t_page_dir)

In [ ]:
index = 0
page_ref_table = []
for page in df[0]:
    for p in range(0,TEMPLATE_DEEP):
        page_template = re.sub("(/[^/]+){"+str(p)+"}$" , "" , page)
        resp = requests.get(page_template)
        html = resp.content.decode("utf-8")
        filepath = page_dir+str(index)+"-"+str(p)+'.html'
        f = open(filepath, 'w' , encoding='utf-8')
        f.write(html)
        f.close
        time.sleep(0.5) #intervallo di tempo per non incorrere in filtri anti DDOS
        page_ref_table.append([page_template , p , filepath])
        print("saved "+filepath)
    index += 1
    print("------------------")

In [ ]:
pages_array = np.array(page_ref_table)    
np.save(t_page_dir+"pages.npy" , pages_array)

In [ ]:
a = np.load(t_page_dir+"pages.npy")

In [ ]:
a